# Preamble

In [2]:
import pandas as pd 
import numpy as np 
import sys
import matplotlib.pyplot as plt 
from IPython.display import display, HTML

def custom_head(self, n=5):
    """
    Custom .head() method for all Pandas DataFrames.
    Automatically displays the first n rows as an HTML table in Jupyter Notebook.
    """
    # Convert the first n rows to an HTML table and display it
    display(HTML(self.iloc[:n].to_html(border=1)))
    # Return the first n rows as a regular DataFrame for further chaining
    return self.iloc[:n]

# Monkey patch the Pandas DataFrame class
pd.DataFrame.head = custom_head

# Chapter 1: Data Merging Basics


Datasets for example

The city of Chicago is ivided into fifty local neighbordooids called wards. We have table with data about the local goernment offices in each ward. In this example, we want to merge the local government data with census data about the popultaion of each ward. 



## Inner Joins 

### The ward data

In [3]:
# Load the .p file 
ward_data = pd.read_pickle("/home/khaijones/Data-Camp/Merging with Pandas Data Sets/ward.p")

# Print the DataFrame
ward_data.head(5)
print(ward_data.shape)

,ward,alderman,address,zip
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649


(50, 4)


This table has 50 rows and 5 rows

### Census data

The census table contains the population of each ward in 2000 and 2010, and that change as a percentage. Additionally, it includes the address for the center of each ward. This table has 50 rows and 6 columns.

In [3]:
#  Load the .p file
census = pd.read_pickle("/home/khaijones/Data-Camp/Merging with Pandas Data Sets/census.p")

#Print the DataFrame 
census.head(10)
print(census.shape)


,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637
5,6,54989,52341,-5%,150 WEST 74TH STREET,60636
6,7,54593,51581,-6%,8549 SOUTH OGLESBY AVENUE,60617
7,8,54039,51687,-4%,1346-1352 EAST 75TH STREET,60649
8,9,52008,51519,-1%,11039-11059 SOUTH WENTWORTH AVENUE,60628
9,10,56613,51535,-9%,10534 SOUTH AVENUE F,46394


(50, 6)


This table has 50 rows and 6 rows

The two tables are related by their ward column. We can merge them together, matching the ward number from each row of the wards table to the ward numbers from the census table. For example, the second ward in the wards table with Alderman Brian Hopkins would be matche with row 2 of the census table where the population in 2000 was 54,361.

### Inner join
The pandas package has an excellent DataFrame method for performing this typ of merge called merge. The merge method takes the first DataFrame, wards, and merges it with the second DataFrame, census. We use the argument to tell the method that we want to merge the two DataFrames on the ward column. Since we listed the wards table first, its columns will appear first in the output, followed by the columns from the census table. 

In [9]:
wards_census = ward_data.merge(census, on = 'ward') 
wards_census.head(5);
print(wards_census.shape)

,ward,alderman,address_x,zip_x,pop_2000,pop_2010,change,address_y,zip_y
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


(50, 9)


#### Suffixes

You may have noticed that the merged table has columns with suffixes of underscore x or y. This is because both the wards and census tables contained address and zip columns. To avoid multiple columns with the same name, they are automatically given a suffix by the merge method.

In [6]:
print(wards_census.columns)

Index(['ward', 'alderman', 'address_x', 'zip_x', 'pop_2000', 'pop_2010',
       'change', 'address_y', 'zip_y'],
      dtype='object')


We can use the suffix argument of the merge method to control this behavior. We provide a tuple where all of the overlapping columns in the left table are given the suffix '_ward', and those of the right table will be given the suffix '_cen'. This makes it easier for us to tell the difference between the columns.

In [8]:
wards_census = ward_data.merge(census, on='ward', suffixes=('_ward','_cen'))
wards_census.head()
print(wards_census.shape)

,ward,alderman,address_ward,zip_ward,pop_2000,pop_2010,change,address_cen,zip_cen
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


(50, 9)


## Practice Questions

(1) What column to merge on? 

Chicago provides a list of taxicab owners and vehicles licensed to operate within the city, for public safety. Your goal is to merge two table together. One table is called 'taxi_owners', with info about the taxi cab company owners, and one is called 'taxi_veh' , with info about ec taxi cab vehicle. Both the 'taxi_owners' and 'taxi_veh' tables have been loaded for you to explore. 

Choose the column you would use to merge the two tables on using the '.merge()' method. 

The 'vid' column

In [11]:
taxi_owners = pd.read_pickle('/home/khaijones/Data-Camp/Merging with Pandas Data Sets/taxi_owners.p')  
taxi_veh = pd.read_pickle("/home/khaijones/Data-Camp/Merging with Pandas Data Sets/taxi_vehicles.p")

taxi_owners.head()
print('      ')
taxi_veh.head();

,rid,vid,owner,address,zip
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659
2,T1495,1495,"FUNRIDE, INC.",3351 W. ADDISON ST.,60618
3,T4231,4231,ALQUSH CORP.,6611 N. CAMPBELL AVE.,60645
4,T5971,5971,EUNIFFORD INC.,3351 W. ADDISON ST.,60618


,vid,make,model,year,fuel_type,owner
0,2767,TOYOTA,CAMRY,2013,HYBRID,SEYED M. BADRI
1,1411,TOYOTA,RAV4,2017,HYBRID,DESZY CORP.
2,6500,NISSAN,SENTRA,2019,GASOLINE,AGAPH CAB CORP
3,2746,TOYOTA,CAMRY,2013,HYBRID,"MIDWEST CAB CO, INC"
4,5922,TOYOTA,CAMRY,2013,HYBRID,SUMETTI CAB CO


(2) My first inner join 

You have been tasked with figuring out what moste popular types of fuel used in Chigago taxis are. To complete the analysis, you need to merg the 'taxi_owners' and 'taxi_veh' tables together on the vid column. You can then use the merged table along with the '.value_counts()' method to find the most common fuel_type. 

In [13]:
# Merge the taxi_owners and taxi_veh tables 
taxi_own_veh = taxi_owners.merge(taxi_veh, on='vid')

# Print columns of DataFrame 
print(taxi_own_veh.columns)

Index(['rid', 'vid', 'owner_x', 'address', 'zip', 'make', 'model', 'year',
       'fuel_type', 'owner_y'],
      dtype='object')


In [18]:
#Assign suffixes to taxi_own_veh

taxi_own_veh = taxi_owners.merge(taxi_veh, on='vid', suffixes = ('_own' , '_veh') )

# Print columns of DataFrame 
print(taxi_own_veh.columns)

Index(['rid', 'vid', 'owner_own', 'address', 'zip', 'make', 'model', 'year',
       'fuel_type', 'owner_veh'],
      dtype='object')


In [19]:
print(taxi_own_veh['fuel_type'].value_counts())

fuel_type
HYBRID                    2792
GASOLINE                   611
FLEX FUEL                   89
COMPRESSED NATURAL GAS      27
Name: count, dtype: int64


## One-to-one relationships

In a one-to-one relationship, every row in the left table is related to one and only row in the right table. Every row in the wards table is related to only one row in the census table, so there is only one row for ward 3 in each table. Practically speaking, it only makes sense that there is one row of population information for each ward. It wouldn't make sense if the census table contained multiple population values in 2000 for the third ward. 

## One to many relationship

In a one-to-many relationship, every row in the left table is related to one or more rows in the right table. To provide an example of a one-to-many relationship, let's think back to our wards table. Within each ward, there are many businesses. We will merge the wards table with a table of licensed businesses in each ward.  The business license data is stored in another table called liceneses. It holds info such as the business address and ward the business is located within. 

In [21]:
# Merge the wards and census tables on the ward column 
wards_census = ward_data.merge(census, on='ward' ) 

# Print the shape of wards_census
print('wards_census table shape:' ,  wards_census.shape) 



wards_census table shape: (50, 9)


In [5]:
license = pd.read_pickle('/home/khaijones/Data-Camp/Merging with Pandas Data Sets/licenses.p')

license.head()
license.shape

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633
2,10002,14,775,CELINA DELI,5089 S ARCHER AVE,60632
3,10005,12,NaN,KRAFT FOODS NORTH AMERICA,2005 W 43RD ST,60609
4,10044,44,638,NEYBOUR'S TAVERN & GRILLE,3651 N SOUTHPORT AVE,60613


(10000, 6)

When we merge the two tables together with the merge method, setting the 'on' attribute to the column ward, the resulting table has both local ward data and business license data. Notice that ward 1 and its alderman Joe is repeated in the resulting table because the licenses table has many businesses in the 1st ward. pandas takes care of the one-to-many relationships for us and doesn't require anything special on our end. We can use the same syntax as we did with one-to-one relationships.

In [6]:
ward_license = ward_data.merge(license, on='ward', suffixes = ('_ward','_license' )) 

ward_license.head();

,ward,alderman,address_ward,zip_ward,account,aid,business,address_license,zip_license
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,12024,NaN,DIGILOG ELECTRONICS,1038 N ASHLAND AVE,60622
1,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,14446,743,EMPTY BOTTLE INC,1035 N WESTERN AVE 1ST,60622
2,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,14624,775,LITTLE MEL'S HOT DOG,2205 N CALIFORNIA AVE,60647
3,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,14987,NaN,MR. BROWN'S LOUNGE,2301 W CHICAGO AVE 1ST,60622
4,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,15642,814,Beat Kitchen,2000-2100 W DIVISION ST,60622


By printing the shape, we can see that our original wards table has 50 rows. After merging the wards table with the licenses table, the resulting table has 10,000 rows. When you merge tables that have a one-to-many relationship, the number of rows returned will likely be different than the number in the left table.

In [7]:
print("Ward's DataFrame's Shape", ward_data.shape) 
print("    ") 
print("Ward_liceneses' Shape:", ward_license.shape)

Ward's DataFrame's Shape (50, 4)
    
Ward_liceneses' Shape: (10000, 9)


A business may have one or multiple owners. In this exercise, you will continue to gain experience with one-to-many merges by merging a table of business owners, called "biz_owners" , to the "licenses" table. Recall from the video lesson, with a one-to-many relationship, a row in the left table may be repeated if it is related to multiple rows in the right table. In this lesson, you will explore this further by finding out what is the most common business owner title. (i.e., secretary, CEO, or vice president)

In [40]:
 biz_owners = pd.read_pickle('/home/khaijones/Data-Camp/Merging with Pandas Data Sets/business_owners.p') 

# Merge the licenses and biz_owners table on account  
licenses_owners  = license.merge(biz_owners, on='account')

# Group the results by title then count the number of accounts 
counted_df = licenses_owners.groupby('title').agg({'account': 'count'}) 

# Sort the counted_df in descening orer 
sorted_df = counted_df.sort_values('account', ascending=False) 

# Use .head() method to print the first few rows of sorted_df 
sorted_df.head();



,account
title,
PRESIDENT,6259
SECRETARY,5205
SOLE PROPRIETOR,1658
OTHER,1200
VICE PRESIDENT,970


## Merging multiple DataFrames

In [46]:
# Review new Data 

grant = pd.read_pickle('/home/khaijones/Data-Camp/Merging with Pandas Data Sets/cta_calendar.p') 

grant.head();


,year,month,day,day_type
0,2019,1,1,Sunday/Holiday
1,2019,1,2,Weekday
2,2019,1,3,Weekday
3,2019,1,4,Weekday
4,2019,1,5,Saturday


### Total riders in a month

Your goal is to find the total number of rides provided to passengers passing through the Wilson station (station_name == 'Wilson') when riding Chicago's public transportation system on weekdays (day_type == 'Weekday') in July (month == 7). Luckily, Chicago provides this detailed data, but it is in three different tables. You will work on merging these tables together to answer the question. This data is different from the business related data you have seen so far, but all the information you need to answer the question is provided.

The cal, ridership, and stations DataFrames have been loaded for you. The relationship between the tables can be seen in the diagram below.

Table diagram. The cal table relates to ridership via year, month, and day. The ridership table relates to the stations table via station_id.

In [ ]:
# Merge the ridership and cal tables
ridership_cal = ridership.merge(cal, on = ['year', 'month', 'day']) \
            				.merge(stations, on = 'station_id')

# Create a filter to filter ridership_cal_stations
filter_criteria = ((ridership_cal_stations['month'] == 7) 
                   & (ridership_cal_stations['day_type'] == 'Weekday') 
                   & (ridership_cal_stations['station_name'] == 'Wilson'))

# Use .loc and the filter to select for rides
print(ridership_cal_stations.loc[filter_criteria, 'rides'].sum())

Three table merge
To solidify the concept of a three DataFrame merge, practice another exercise. A reasonable extension of our review of Chicago business data would include looking at demographics information about the neighborhoods where the businesses are. A table with the median income by zip code has been provided to you. You will merge the licenses and wards tables with this new income-by-zip-code table called zip_demo.

The licenses, wards, and zip_demo DataFrames have been loaded for you.

-  Instructions:
  
Starting with the licenses table, merge to it the zip_demo table on the zip column. Then merge the resulting table to the wards table on the ward column. Save result of the three merged tables to a variable named licenses_zip_ward.
Group the results of the three merged tables by the column alderman and find the median income.

In [ ]:
# Merge licenses and zip_demo, on zip; and merge the wards on ward
licenses_zip_ward = licenses.merge(zip_demo, on='zip')\
            		.merge(wards, on='ward')

# Print the results by alderman and show median income
print(licenses_zip_ward.groupby('alderman').agg({'income':'median'}))

### One-to-many merge with multiple tables

In this exercise, assume that you are looking to start a business in the city of Chicago. Your perfect idea is to start a company that uses goats to mow the lawn for other businesses. However, you have to choose a location in the city to put your goat farm. You need a location with a great deal of space and relatively few businesses and people around to avoid complaints about the smell. You will need to merge three tables to help you choose your location. The land_use table has info on the percentage of vacant land by city ward. The census table has population by ward, and the licenses table lists businesses by ward.

- Merge land_use and census on the ward column. Merge the result of this with licenses on the ward column, using the suffix _cen for the left table and _lic for the right table. Save this to the variable land_cen_lic.

In [ ]:
# Merge land_use and census and merge result with licenses including suffixes
land_cen_lic = land_use.merge(census, on='ward') \
               .merge(licenses, on='ward', suffixes=['_cen', '_lic'])

- Group land_cen_lic by ward, pop_2010 (the population in 2010), and vacant, then count the number of accounts. Save the results to pop_vac_lic.

In [ ]:
# Merge land_use and census and merge result with licenses including suffixes
land_cen_lic = land_use.merge(census, on='ward') \
                    .merge(licenses, on='ward', suffixes=('_cen','_lic'))

# Group by ward, pop_2010, and vacant, then count the # of accounts
pop_vac_lic = land_cen_lic.groupby(['ward', 'pop_2010', 'vacant'], 
                                   as_index=False).agg({'account':'count'})

- Sort pop_vac_lic by vacant, account, andpop_2010 in descending, ascending, and ascending order respectively. Save it as sorted_pop_vac_lic.

In [ ]:
# Merge land_use and census and merge result with licenses including suffixes
land_cen_lic = land_use.merge(census, on='ward') \
                    .merge(licenses, on='ward', suffixes=('_cen','_lic'))

# Group by ward, pop_2010, and vacant, then count the # of accounts
pop_vac_lic = land_cen_lic.groupby(['ward','pop_2010','vacant'], 
                                   as_index=False).agg({'account':'count'})

# Sort pop_vac_lic and print the results
sorted_pop_vac_lic = pop_vac_lic.sort_values(['vacant', 'account', 'pop_2010'], 
                                             ascending=[False, True, True])

# Print the top few rows of sorted_pop_vac_lic
print(sorted_pop_vac_lic.head())

# Chapter 2: Mergining tables with different Joint Types

### Left Join

In chapter 1, we introduced the pandas merge method that allows us to combine two tables by specifying one or more key columns to link the tables by. By default, the merge method performs an inner join, returning only the rows of data with matching values in the key columns of both tables.

In this lesson, we'll talk about the idea of a left join. A left join returns all rows of data from the left table and only those rows from the right table where key columns match.

Here we have two tables named left and right. We want to use a left join to merge them on key column C. A left join returns all of the rows from the left table and only those rows from the right table where column C matches in both. Notice the second row of the merged table. The columns from the left table are filled in, while the column from the right table is not since there wasn't a match found for that row in the right table. Let's review another example.



### New Dataset

To help us learn more about left joins and other concepts n this chapter, we will use data from The Movie Database, a community-built movie database with infor on thousands of movies, their casts, and popularity. In our next example, we have two tables from The Movie Databane that we want to merge. Our first table, named movies, holds information about individual movies such as the title name and it's popularity. Additionally, each movie is given an ID number. Our table starts with 4,803 rows of data.

In [19]:
movies = pd.read_pickle('/home/khaijones/Data-Camp/Merging with Pandas Data Sets/movies.p') 

movies.head()
print(movies.shape)

,id,title,popularity,release_date
0,257,Oliver Twist,20.415572,2005-09-23
1,14290,Better Luck Tomorrow,3.877036,2002-01-12
2,38365,Grown Ups,38.864027,2010-06-24
3,9672,Infamous,3.680896,2006-11-16
4,12819,Alpha and Omega,12.300789,2010-09-17


(4803, 4)


### Tagline table

Or second table is named taglines, which contains a movie ID number and the tag line for the movie. Notice that this tabl has almost 4,000 rows of data, so it contains fewer movies than the movies table.

In [10]:
taglines = pd.read_pickle('/home/khaijones/Data-Camp/Merging with Pandas Data Sets/taglines.p')

taglines.head() 
print(taglines.shape)

,id,tagline
0,19995,Enter the World of Pandora.
1,285,"At the end of the world, the adventure begins."
2,206647,A Plan No One Escapes
3,49026,The Legend Ends
4,49529,"Lost in our world, found in another."


(3955, 2)


### Merge with left join

To merge these two tables with a left join, we use our merge method similar to what we learned in chapter 1. Here we list the movie table first and merge it to the taglines table on the ID column in both tables. However, notice an additional argument named 'how'. This argument defines how to merge the two tables. In this case, we use 'left' for a left join. The default value for how is 'inner', so we didn't need to specify this in Chapter 1 since we were only working with inner joins. The result of the merge shows a table with all of the rows from the movies table and a value for tag line where the ID column matches in both tables. Wherever there isn't a matching ID in the taglines table, a null value is entered for the tag line. Remember that pandas uses NaN to denote missing data.

In [14]:
movies_taglines = movies.merge(taglines, on='id', how= 'left') 
movies_taglines.head();

,id,title,popularity,release_date,tagline
0,257,Oliver Twist,20.415572,2005-09-23,NaN
1,14290,Better Luck Tomorrow,3.877036,2002-01-12,Never underestimate an overachiever.
2,38365,Grown Ups,38.864027,2010-06-24,Boys will be boys. . . some longer than others.
3,9672,Infamous,3.680896,2006-11-16,There's more to the story than you know
4,12819,Alpha and Omega,12.300789,2010-09-17,A Pawsome 3D Adventure


In [15]:
print(movies_taglines.shape)

(4803, 5)


After the merge, our resulting table has 4,803 rows. This is because we are returning all of the rows of data from the movies table, and the relationship between the movies table and taglines is one-to-one. Therefore, in a one-to-one merge like this one, a left join will always return the same number rows as the left table.

In [ ]:
## Other Joins 

### Right Join 

The right will return all of the rows from the right table and includes only those rows from the left table that have matching values. It is the mirror opposite of the left join. Only rows from the 
left table where the column matches are returned. Where there isn't a match, the columns from the left table will be missing in the result table, like rows one and four. Let's look at another table called movie_to_genres. Movies
can have multple genres, and this table lists different genres for each movie. 

In [10]:
movie_to_genres =  pd.read_pickle('/home/khaijones/Data-Camp/Merging with Pandas Data Sets/movie_to_genres.p')
tv_genre = movie_to_genres[movie_to_genres['genre'] == 'TV Movie'] 
tv_genre.head(6);

,movie_id,genre
4998,10947,TV Movie
5994,13187,TV Movie
7443,22488,TV Movie
10061,78814,TV Movie
10790,153397,TV Movie
10835,158150,TV Movie
11096,205321,TV Movie
11282,231617,TV Movie


For our right join example, let's take a sample of this data subsetting to develop a table of movies from the TV Movie genre. Our goal is to merge it with the movies table. We will set movies as our left table and merge it with the tv_genre table. we want to use a right join to chek that our movies table is not missing data. In additon to showing a right join, this example also allows us to look at another feature. Notice that the column with the movie ID number in the movies table is named id, an in the tv_genre table is is named movie_id. The merge methoud has a f

In [21]:
tv_movies = movies.merge(tv_genre, how='right', left_on='id', right_on='movie_id') 

tv_movies.head();

,id,title,popularity,release_date,movie_id,genre
0,10947,High School Musical,16.536374,2006-01-20,10947,TV Movie
1,13187,A Charlie Brown Christmas,8.701183,1965-12-09,13187,TV Movie
2,22488,Love's Abiding Joy,1.128559,2006-10-06,22488,TV Movie
3,78814,We Have Your Husband,0.102003,2011-11-12,78814,TV Movie
4,153397,Restless,0.812776,2012-12-07,153397,TV Movie


The code for this merge has some new elements. First of all, we set the how argument to right so that the merge performs a right join. Additionally, we introduce two new arguments, named left_on and right_on. They allow us to tell the merge which key columns from each table to merge the tables. We list movies as the left table, so we set left_on to id and right_on to movie_id. Our returned table has movies that match our table of tv_genres. There does not appear to be any null values in the columns from the movies table. We could explore further. However, let's move on to our last type of join.

### Outer Join 

The last type of join is called an aoutre join. An outer join will return all of the rows from both tables regardless if there is a match between the tables. Where the key column used to join the tables has no match, null values are returned.  For an example of ths, we filter the movie_to_genres tabble as before into two very small tables. One table has data on Family movies, and the other has Comedy movies. 

In [22]:
m = movie_to_genres['genre'] == 'Family' 
family = movie_to_genres[m].head(3) 


,movie_id,genre
5,12,Family
33,35,Family
111,105,Family


In [23]:
m = movie_to_genres['genre'] == 'Comedy' 
comedy = movie_to_genres[m].head(3)

,movie_id,genre
1,5,Comedy
7,13,Comedy
35,35,Comedy


### Merge with outer join 

In this merge, we list the family table as the left table and merge it on the movie_id column. The how argument is set to outer for an outer join. Both of our tables have the same column names. Therefore, we add suffixes to show what table the columns originated. In our result table, every row is returned for both tables and we see some null values. In our original comedy tables ID number 12 does not exist. Therefore a null is shown. Similarly, in our last row, movie ID 13 wasn't in the family dataset so it has a null.

In [24]:
family_comedy = family.merge(comedy, on='movie_id', how='outer', suffixes=('_fam', '_com')) 

print(family_comedy)

   movie_id genre_fam genre_com
0         5       NaN    Comedy
1        12    Family       NaN
2        13       NaN    Comedy
3        35    Family    Comedy
4       105    Family       NaN


In [ ]:
# Use merge_ordered() to merge gdp and sp500 on year and date
gdp_sp500 = pd.merge_ordered(gdp, sp500, left_on='year', right_on='date', 
                             how='left')

# Print gdp_sp500b
print(gdp_sp500)

In [ ]:
# Use merge_ordered() to merge inflation, unemployment with inner join
inflation_unemploy = pd.merge_ordered(inflation, unemployment, on='date', how='inner')

# Print inflation_unemploy 
print(inflation_unemploy)

# Plot a scatter plot of unemployment_rate vs cpi of inflation_unemploy
inflation_unemploy.plot(x='unemployment_rate', y='cpi', kind='scatter')
plt.show()